# Logistic model

In [12]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import ipywidgets as widgets
from IPython.display import display

# --------------------------------------------------
# 1. Load Data and Train the Model
# --------------------------------------------------
def load_data_and_train_model():
    # Load Excel files (make sure these files are in your Colab session)
    msk_df = pd.read_excel("msk_cleaned.xlsx")
    pbs_df = pd.read_excel("pbs_cleaned.xlsx")

    # Pivot the PBS DataFrame so that each risk factor becomes its own column.
    # The unique PBS values are: "PBS28", "PBS52", "PBS68", "PBS8", "PBS84"
    pbs_pivot = pbs_df.pivot_table(
        index=["year", "age_group", "gender"],
        columns="pbs",
        values="crude rate of patients",
        aggfunc="sum"
    ).reset_index()

    # For consistency, rename columns to uppercase names.
    pbs_pivot.rename(columns={"year": "Year", "age_group": "Age_group", "gender": "Gender"}, inplace=True)
    msk_df.rename(columns={"year": "Year", "age_group": "Age_group",
                           "gender": "Gender", "crude rate of MSK patients": "MSK_rate"}, inplace=True)

    # Merge the MSK disease data with the PBS risk factors on Year, Age_group, and Gender.
    merged_df = pd.merge(msk_df, pbs_pivot, on=["Year", "Age_group", "Gender"], how="inner")

    # Create a binary outcome:
    # If MSK_rate exceeds its upper quantile value, label as 1 (high risk), otherwise label as 0.
    threshold = merged_df["MSK_rate"].quantile(0.7)
    merged_df["disease_binary"] = (merged_df["MSK_rate"] > threshold).astype(int)

    # Define risk factor columns (from the PBS data)
    risk_factors = ["PBS28", "PBS52", "PBS68", "PBS8", "PBS84"]
    merged_df[risk_factors] = merged_df[risk_factors].fillna(0)

    # Define predictors: risk factors plus demographic columns.
    predictors = risk_factors + ["Year", "Age_group", "Gender"]
    X = merged_df[predictors]
    y = merged_df["disease_binary"]

    # Preprocessing:
    # - Standardize numeric risk factor values.
    # - One-hot encode categorical variables (dropping the first category to avoid multicollinearity).
    categorical_features = ["Year", "Age_group", "Gender"]
    numeric_features = risk_factors

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), numeric_features),
            ("cat", OneHotEncoder(drop="first"), categorical_features)
        ]
    )

    # Build a pipeline that first preprocesses the data then fits a logistic regression model.
    model_pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("classifier", LogisticRegression(solver="lbfgs", max_iter=1000))
    ])

    # Fit the logistic regression model.
    model_pipeline.fit(X, y)

    return model_pipeline, risk_factors

# Train the model
model_pipeline, risk_factor_cols = load_data_and_train_model()

# --------------------------------------------------
# 2. Define a Prediction Function
# --------------------------------------------------
def predict_msk(PBS28, PBS52, PBS68, PBS8, PBS84, Year, Age_group, Gender):
    """
    Construct a single observation DataFrame from the inputs,
    and use the trained model_pipeline to predict the probability
    of high MSK disease risk.
    """
    input_data = pd.DataFrame({
        "PBS28": [PBS28],
        "PBS52": [PBS52],
        "PBS68": [PBS68],
        "PBS8": [PBS8],
        "PBS84": [PBS84],
        "Year": [Year],
        "Age_group": [Age_group],
        "Gender": [Gender]
    })

    # Get probability for class 1 (high risk)
    prob = model_pipeline.predict_proba(input_data)[:, 1][0]
    print(f"Predicted probability of high MSK disease risk: {prob:.2%}")

# --------------------------------------------------
# 3. Create Interactive Widgets Using ipywidgets
# --------------------------------------------------
# For numeric inputs (risk factors), use BoundedFloatText to restrict the range.
PBS28_widget = widgets.BoundedFloatText(value=0.0, min=0.0, max=100000, description="PBS28:")
PBS52_widget = widgets.BoundedFloatText(value=0.0, min=0.0, max=100000, description="PBS52:")
PBS68_widget = widgets.BoundedFloatText(value=0.0, min=0.0, max=100000, description="PBS68:")
PBS8_widget  = widgets.BoundedFloatText(value=0.0, min=0.0, max=100000, description="PBS8:")
PBS84_widget = widgets.BoundedFloatText(value=0.0, min=0.0, max=100000, description="PBS84:")

# For categorical inputs, define dropdown widgets.
Year_widget = widgets.Dropdown(
    options=["2011–12", "2012–13", "2013–14", "2014–15", "2015–16", "2016–17", "2017–18", "2018–19"],
    value="2013–14",
    description="Year:"
)
Age_group_widget = widgets.Dropdown(
    options=["<25", "25-29", "30-39", "40-49", "50-59", "60-69", "70-79", "80+"],
    value="40-49",
    description="Age Group:"
)
Gender_widget = widgets.Dropdown(
    options=["Males", "Females"],
    value="Males",
    description="Gender:"
)

# Organize widgets into a user-friendly layout.
ui = widgets.VBox([
    widgets.VBox([PBS28_widget, PBS52_widget, PBS68_widget, PBS8_widget, PBS84_widget]),
    Year_widget,
    Age_group_widget,
    Gender_widget
])

# Connect widgets to the predict_msk function.
out = widgets.interactive_output(
    predict_msk,
    {
        "PBS28": PBS28_widget,
        "PBS52": PBS52_widget,
        "PBS68": PBS68_widget,
        "PBS8": PBS8_widget,
        "PBS84": PBS84_widget,
        "Year": Year_widget,
        "Age_group": Age_group_widget,
        "Gender": Gender_widget,
    }
)

# Display the interactive form and the output.
display(ui, out)


Output()

# Linear model

In [14]:
# --------------------------------------------------
# 1. Import necessary libraries
# --------------------------------------------------
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import ipywidgets as widgets
from IPython.display import display

# --------------------------------------------------
# 2. Load Data and Train the Linear Regression Model (without "Year")
# --------------------------------------------------
def load_data_and_train_linear_model():
    # Load Excel files (ensure these files are available in your environment)
    msk_df = pd.read_excel("msk_cleaned.xlsx")
    pbs_df = pd.read_excel("pbs_cleaned.xlsx")

    # Drop "year" column from both dataframes if they exist
    if "year" in msk_df.columns:
        msk_df.drop(columns=["year"], inplace=True)
    if "year" in pbs_df.columns:
        pbs_df.drop(columns=["year"], inplace=True)

    # Pivot the PBS DataFrame so that each risk factor becomes its own column.
    # The unique PBS values are: "PBS28", "PBS52", "PBS68", "PBS8", "PBS84"
    pbs_pivot = pbs_df.pivot_table(
        index=["age_group", "gender"],
        columns="pbs",
        values="crude rate of patients",
        aggfunc="sum"
    ).reset_index()

    # Rename for consistency
    pbs_pivot.rename(columns={"age_group": "Age_group", "gender": "Gender"}, inplace=True)
    msk_df.rename(columns={
        "age_group": "Age_group",
        "gender": "Gender",
        "crude rate of MSK patients": "MSK_rate"
    }, inplace=True)

    # Merge the MSK data with the PBS risk factors on Age_group and Gender.
    merged_df = pd.merge(msk_df, pbs_pivot, on=["Age_group", "Gender"], how="inner")

    # Fill missing PBS values with 0
    risk_factors = ["PBS28", "PBS52", "PBS68", "PBS8", "PBS84"]
    merged_df[risk_factors] = merged_df[risk_factors].fillna(0)

    # Define predictors and outcome.
    # Predictors now include the risk factors and demographic columns *[Age_group, Gender]*.
    predictors = risk_factors + ["Age_group", "Gender"]
    X = merged_df[predictors]
    y = merged_df["MSK_rate"]

    # Preprocessing steps:
    # - Standardize numeric risk factor variables.
    # - One-hot encode the categorical variables (dropping the first to avoid multicollinearity)
    categorical_features = ["Age_group", "Gender"]
    numeric_features = risk_factors

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), numeric_features),
            ("cat", OneHotEncoder(drop="first"), categorical_features)
        ]
    )

    # Build a pipeline to preprocess the data and fit the linear regression model.
    model_pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("regressor", LinearRegression())
    ])

    # Fit the model.
    model_pipeline.fit(X, y)

    # Compute and print the training R² score.
    r2_score = model_pipeline.score(X, y)
    # print(f"Training R² Score: {r2_score:.4f}")

    return model_pipeline, risk_factors

# Train the linear regression model and get risk factor columns.
model_pipeline, risk_factor_cols = load_data_and_train_linear_model()

# --------------------------------------------------
# 3. Define the Prediction Function (without "Year")
# --------------------------------------------------
def predict_msk_linear(PBS28, PBS52, PBS68, PBS8, PBS84, Age_group, Gender):
    """
    Build a one-row DataFrame from the user inputs and predict
    the crude rate of MSK disease (per 100,000 people) using the trained model.
    """
    input_data = pd.DataFrame({
        "PBS28": [PBS28],
        "PBS52": [PBS52],
        "PBS68": [PBS68],
        "PBS8":  [PBS8],
        "PBS84": [PBS84],
        "Age_group": [Age_group],
        "Gender": [Gender]
    })

    # Predict the outcome.
    pred = model_pipeline.predict(input_data)[0]
    print(f"Predicted crude rate of MSK disease (per 100,000): {pred:.2f}")

# --------------------------------------------------
# 4. Create Interactive Widgets Using ipywidgets (without Year)
# --------------------------------------------------
# Widgets for risk factors (using BoundedFloatText to restrict input)
PBS28_widget = widgets.BoundedFloatText(value=0.0, min=0.0, max=100000, description="PBS28:")
PBS52_widget = widgets.BoundedFloatText(value=0.0, min=0.0, max=100000, description="PBS52:")
PBS68_widget = widgets.BoundedFloatText(value=0.0, min=0.0, max=100000, description="PBS68:")
PBS8_widget  = widgets.BoundedFloatText(value=0.0, min=0.0, max=100000, description="PBS8:")
PBS84_widget = widgets.BoundedFloatText(value=0.0, min=0.0, max=100000, description="PBS84:")

# Dropdowns for categorical variables (Age_group and Gender)
Age_group_widget = widgets.Dropdown(
    options=["<25", "25-29", "30-39", "40-49", "50-59", "60-69", "70-79", "80+"],
    value="40-49",
    description="Age Group:"
)
Gender_widget = widgets.Dropdown(
    options=["Males", "Females"],
    value="Males",
    description="Gender:"
)

# Arrange the widgets into a neat layout.
ui = widgets.VBox([
    widgets.VBox([PBS28_widget, PBS52_widget, PBS68_widget, PBS8_widget, PBS84_widget]),
    Age_group_widget,
    Gender_widget
])

# Connect the widgets to the prediction function.
out = widgets.interactive_output(
    predict_msk_linear,
    {
        "PBS28": PBS28_widget,
        "PBS52": PBS52_widget,
        "PBS68": PBS68_widget,
        "PBS8": PBS8_widget,
        "PBS84": PBS84_widget,
        "Age_group": Age_group_widget,
        "Gender": Gender_widget,
    }
)

# Display the interactive UI along with live output.
display(ui, out)


Output()